In [2]:
%run InitialSetup.ipynb

In [3]:
# Construct Surface between 5X5 and 20x20 MaturityXTenor tuples
it_range = [i for i in range(5, 11)] + [15, 20]

# Create tuples of dates
iter_list = list(itertools.product(*[it_range, it_range]))

# Create similar names to the titles of the vol_data file
vol_filt = [str(i)+'Y '+str(j)+'Y' for i, j in iter_list]

# Filter vol_data based on criteria
vol_surface = vol_data[vol_filt]

In [4]:
# Vectorize functions
Hagan_Vect = np.vectorize(sigma_CEV_Hagan)
Black_Vect = np.vectorize(payer_Black)
Chi_Square_Vect = np.vectorize(payer_chi_square)

In [5]:
# Set data as an array 
row_wise_vol_data = np.array(vol_surface.values.flatten())

# Define list of maturities and tenors
eval_dates = [DateParser.parseFormatted(datetime.strftime(date,'%Y-%m-%d'), '%Y-%m-%d') for date in vol_surface.index]
eval_dates = np.array(eval_dates)
full_eval_dates = np.repeat(eval_dates, vol_surface.shape[1])

#Repeat each date for each swaption we selected in vol_surface
maturity_array = np.array([item[1] for item in iter_list]*len(vol_data))
maturities = [date + Period(int(i), Years) for date, i in zip(full_eval_dates, maturity_array)]
maturities = np.array(maturities)
#tenor_array = np.array([item[0] for item in iter_list]*len(vol_data))

#Forward Rates to be added

#Construct the full calibration data
full_calib_data = np.matrix(np.stack([full_eval_dates,maturities, row_wise_vol_data], axis = 1))

In [6]:
# Find optimal sigma in Hagan Approx 
S0 = 0.035
K = 0.035
sigma = 0.09999978695526811
sigma2 = 0.07
beta = 0.35

#Test0 = sigma and Test1 = beta
# Construct the function to be optimized
opt_func = lambda test: np.sum(np.square(Hagan_Vect(S0, K ,test[0], test[1], 
                                                full_calib_data[:3000,0], 
                                                full_calib_data[:3000,1]) 
                                                - full_calib_data[:3000,2]))

In [ ]:
'''OPTIMIZATION SCHEMES'''
'''1) Newton's Method'''
# optimize.newton(opt_func, [0.01, 0.5], tol = 1e-5, maxiter=100000)

In [ ]:
'''2) SHGO Algorithm'''
start_opti = time.time()
optimize.shgo(opt_func, bounds = [(0, 1), (0, 1)], n=1000,iters = 1e+9, 
              sampling_method='sobol',
             options = {'ftol': 1e-9})
end_opti = time.time()

In [ ]:
'''3) Least Squares Optimization'''
start_LSO = time.time()
optimize.least_squares(opt_func, x0 = [0.01,0.5], bounds = [(0, 1),(0, 1)],
                       ftol = 1e-9, 
                       xtol=1e-09, 
                       gtol=1e-09)
end_LSO = time.time()

In [ ]:
'''4) Dual Annealing Optimization'''
start_DAO = time.time()
optimize.least_squares(opt_func, bounds = [(0, 1),(0, 1)],
                       maxiter = 1e+9, 
                       xtol=1e-09, 
                       gtol=1e-09)
end_DAO = time.time()

In [ ]:
'''5) Fmin tnc Optimization'''
start_TNC = time.time()
optimize.fmin_tnc(opt_func, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  approx_grad = True,
                  epsilon= 1e-9)
end_TNC = time.time()

In [ ]:
'''6) Fmin l_bfgs_b Optimization'''
start_BFGS = time.time()
optimize.fmin_l_bfgs_b(opt_func, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  approx_grad = True,
                  m = 1e+9)
end_BFGS = time.time()

In [ ]:
'''7) Powell Optimization'''
start_Pow = time.time()
optimize.fmin_powell(opt_func, 
                  x0 = [0.01,0.5],
                  maxiter = 1e+9, 
                  xtol=1e-09, 
                  ftol=1e-09)
end_Pow = time.time()

In [ ]:
'''8) Differential Evolution Optimization'''
start_DEO = time.time()
optimize.fmin_powell(opt_func, 
                  bounds = [(0, 1),(0, 1)],
                  maxiter = 1e+9, 
                  tol=1e-09)
end_DEO = time.time()

In [ ]:
'''9) Basinhopping Optimization'''
start_BHO = time.time()
optimize.basinhopping(opt_func, 
                  x0 = [0.01,0.5],
                  niter = 1e+9)
end_BHO = time.time()

In [ ]:
'''10) SLSQP Optimization'''
start_SLSQP = time.time()
optimize.fmin_slsqp(opt_func, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  iter = 1e+9)
end_SLSQP = time.time()